In [173]:
#imports
import os
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor, Normalize
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from tqdm import tqdm as tqdm
import torch.optim.lr_scheduler as lr_scheduler

In [174]:
#directories
BDSL_TRAIN = 'E:/Bangla Sign Alphabet/Bangla Sign Alphabet Data/BDSL/BDSL_split/train'
BDSL_VAL = 'E:/Bangla Sign Alphabet/Bangla Sign Alphabet Data/BDSL/BDSL_split/val'
BDSL_TEST = 'E:/Bangla Sign Alphabet/Bangla Sign Alphabet Data/BDSL/BDSL_split/test'

ISHARA_AUGMENETED_TRAIN = 'E:/Bangla Sign Alphabet/Bangla Sign Alphabet Data/Isharalipi_augmentated/Isharalipi_augmentated_split/train'
ISHARA_AUGMENETED_VAL = 'E:/Bangla Sign Alphabet/Bangla Sign Alphabet Data/Isharalipi_augmentated/Isharalipi_augmentated_split/val'
ISHARA_AUGMENETED_TEST = 'E:/Bangla Sign Alphabet/Bangla Sign Alphabet Data/Isharalipi_augmentated/Isharalipi_augmentated_split/test'

ISHARA_MAIN = 'E:/Bangla Sign Alphabet/Bangla Sign Alphabet Data/Isharalipi_original/Isharalipi_original_main'

In [175]:
TRAIN_DIRECTORY = ISHARA_AUGMENETED_TRAIN
VAL_DIRECTORY = ISHARA_AUGMENETED_VAL
TEST_DIRECTORY = BDSL_TEST

In [176]:
print(os.listdir(TRAIN_DIRECTORY))

['10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44']


In [177]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [178]:
# Hyper-parameters 
num_epochs = 15
batch_size = 100
learning_rate = 0.01

In [179]:
img_transform = transforms.Compose([transforms.Resize((64,64)), transforms.ToTensor()])

train_data = ImageFolder(TRAIN_DIRECTORY, transform=img_transform)
val_data = ImageFolder(VAL_DIRECTORY, transform=img_transform)
test_data = ImageFolder(TEST_DIRECTORY, transform=img_transform)

In [180]:
print(len(train_data))
print(len(val_data))
print(len(test_data))

18968
2353
2416


In [181]:
train_dl = DataLoader(train_data, batch_size, shuffle=True, num_workers=4, pin_memory=True)
test_dl = DataLoader(test_data, batch_size*2, num_workers=4, pin_memory=True)
val_dl = DataLoader(val_data, batch_size*2,shuffle=True, num_workers=4, pin_memory=True)

In [182]:
#load pretrained model
import torchvision.models as models

#model = models.resnet18(pretrained=False)
#model = models.alexnet(pretrained=False)
#model = models.squeezenet1_0(pretrained=False)
#model = models.vgg16(pretrained=False)
#model = models.googlenet(pretrained=True)
#model = models.shufflenet_v2_x1_0(pretrained=False)
#model = models.mobilenet_v2(pretrained=False)
#model = models.resnext50_32x4d(pretrained=False)
#model = models.wide_resnet50_2(pretrained=False)
model = models.mnasnet1_0(pretrained=True)

#in_features
resnet18 = 1000
alexnet = 256*6*6
#squeezenet1_0 = 
vgg16 = 512*7*7
googlenet = 1024*1*1
shufflenet_v2_x1_0 = 1000
mobilenet_v2 = 1280
resnext50_32x4d = 1000
wide_resnet50_2 = 1000
mnasnet1_0 = 1280

In [183]:
for param in model.parameters():
    param.requires_grad = True

model.classifier = nn.Sequential(nn.Linear(in_features=mnasnet1_0, out_features=256, bias=True),
                                 nn.ReLU(inplace=True),
                                 nn.BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True),
                                 nn.Linear(in_features=256, out_features=128, bias=True),
                                 nn.ReLU(inplace=True),
                                 nn.BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True),
                                 nn.Linear(in_features=128, out_features=35, bias=True))

model.to(device)

MNASNet(
  (layers): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.00029999999999996696, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.00029999999999996696, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (7): BatchNorm2d(16, eps=1e-05, momentum=0.00029999999999996696, affine=True, track_running_stats=True)
    (8): Sequential(
      (0): _InvertedResidual(
        (layers): Sequential(
          (0): Conv2d(16, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(48, eps=1e-05, momentum=0.00029999999999996696, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(48, 

In [184]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [185]:
n_total_steps = len(train_dl)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_dl):
    
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

print('Finished Training')

Epoch [1/15], Step [100/190], Loss: 2.9025
Epoch [2/15], Step [100/190], Loss: 1.9229
Epoch [3/15], Step [100/190], Loss: 0.8579
Epoch [4/15], Step [100/190], Loss: 0.6381
Epoch [5/15], Step [100/190], Loss: 0.3715
Epoch [6/15], Step [100/190], Loss: 0.2302
Epoch [7/15], Step [100/190], Loss: 0.1413
Epoch [8/15], Step [100/190], Loss: 0.1237
Epoch [9/15], Step [100/190], Loss: 0.0930
Epoch [10/15], Step [100/190], Loss: 0.0908
Epoch [11/15], Step [100/190], Loss: 0.0452
Epoch [12/15], Step [100/190], Loss: 0.0452
Epoch [13/15], Step [100/190], Loss: 0.0398
Epoch [14/15], Step [100/190], Loss: 0.0374
Epoch [15/15], Step [100/190], Loss: 0.0408
Finished Training


In [186]:
def check_acc(dl):
    n_correct = 0
    n_samples = 0
    model.eval()

    with torch.no_grad():
        for images, labels in dl:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            # max returns (value ,index)
            _, predicted = torch.max(outputs, 1)

            n_correct += (predicted == labels).sum()
            n_samples += predicted.size(0)

        print(f"acc : Got {n_correct} / {n_samples} with accuracy {float(n_correct)/float(n_samples)*100:.2f}")

In [187]:
check_acc(train_dl)

acc : Got 7149 / 18968 with accuracy 37.69


In [188]:
check_acc(test_dl)

acc : Got 310 / 2416 with accuracy 12.83


In [189]:
check_acc(val_dl)

acc : Got 900 / 2353 with accuracy 38.25
